In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dir = '/content/drive/MyDrive/NhanDienCamXuc/images/train'
valid_dir = '/content/drive/MyDrive/NhanDienCamXuc/images/valid'

In [ ]:
# Tham số model
IMG_SIZE = (48, 48)
BATCH_SIZE = 64
EPOCHS = 30
CHANNELS = 1 if len(os.listdir(train_dir + '/angry')[0].endswith('png') else 3  # Xác định grayscale/RGB


In [ ]:
# Tạo Data Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
    shear_range=0.1,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Tạo data loader
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode='grayscale' if CHANNELS == 1 else 'rgb',
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    color_mode='grayscale' if CHANNELS == 1 else 'rgb',
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)

In [ ]:
# Xây dựng model CNN tối ưu cho 48x48
from tensorflow.keras import layers, models

model = models.Sequential([
    # Block 1
    layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(48,48,CHANNELS)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),
    
    # Block 2
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.35),
    
    # Block 3
    layers.Conv2D(256, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.45),
    
    # Fully Connected
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(7, activation='softmax')
])

In [ ]:
# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
# Callbacks
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping, 
    ReduceLROnPlateau
)

callbacks = [
    ModelCheckpoint(
        'best_emotion_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-7
    )
]

In [ ]:
# Training
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_steps=valid_generator.samples // BATCH_SIZE
)


In [ ]:
# Visualize kết quả
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Progress')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Progress')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Đánh giá model
best_model = tf.keras.models.load_model('best_emotion_model.h5')
val_loss, val_acc = best_model.evaluate(valid_generator)
print(f'\nValidation Accuracy: {val_acc*100:.2f}%')
print(f'Validation Loss: {val_loss:.4f}')

# Lưu model cuối cùng
model.save('emotion_model_48x48.h5')

In [ ]:
files.download('emotion_model_48x48.h5')

In [ ]:
files.download('best_emotion_model.h5')